 ---
## 8.&nbsp; Challenge 😃
Now that you've learnt how to send and retrieve information, it's your turn to show off your skills. Create multiple tables in SQL for the data you scrapped about cities from Wikipedia. One should just be a table about the cities, the others should be facts about the cities.

| city_id | city |
| --- | --- |
| 1 | Berlin |
| 2 | Hamburg |
| 3 | Munich |

<br>

| City ID | Population | Year Data Retrieved |
|---|---|---|
| 1 | 3,850,809 | 2024 |
| 2 | 1,945,532 | 2024 |
| 3 | 1,512,491 | 2024 |

> **Pro Tip:** Visualise your relational database with pen and paper before you start coding. This can help you to identify any potential problems or inconsistencies in your design, and it can also make the coding process more efficient.

In [1]:
#!pip install sqlalchemy

**import necessary modules in a Python script**

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# function for webscraping

In [3]:


def cities_dataframe(cities):
  cities_data = []

  for city in cities:
    city_data = {}

    # city
    city_data["City"] = city

    # create the soup
    url = f"https://www.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # country
    city_data["Country"] = city_soup.find(class_="infobox-data").get_text()

    # population
    city_population = city_soup.find(string="Population").find_next("td").get_text()
    city_population_clean = int(city_population.replace(",", ""))
    city_data["Population"] = city_population_clean

    # latitude and longitude
    city_data["Latitude"] = city_soup.find(class_="latitude").get_text()
    city_data["Longitude"] = city_soup.find(class_="longitude").get_text()
    
    #Time_of_retrieve
    current_date = datetime.now()
    city_data['Year Data Retrieved'] = current_date.year
    

    # append this city's data to the cities list
    cities_data.append(city_data)

  return pd.DataFrame(cities_data)

**This code calls a function cities_dataframe() with a list of city names as an argument, creates a DataFrame, and assigns it to the variable city_df. Then, it likely displays the content of the DataFrame city_df**

In [4]:
city_df = cities_dataframe(["Berlin", "Hamburg", "Munich", "Cologne", "Düsseldorf", "Münster", "Hörstel"])
city_df

,City,Country,Population,Latitude,Longitude,Year Data Retrieved
0,Berlin,Germany,3576873,52°31′12″N,13°24′18″E,2024
1,Hamburg,Germany,1945532,53°33′N,10°00′E,2024
2,Munich,Germany,1512491,48°08′15″N,11°34′30″E,2024
3,Cologne,Germany,1073096,50°56′11″N,6°57′10″E,2024
4,Düsseldorf,Germany,619477,51°14′N,6°47′E,2024
5,Münster,Germany,317713,51°57′45″N,07°37′32″E,2024
6,Hörstel,Germany,20506,52°17′50″N,7°35′10″E,2024


In [5]:
city_unique = city_df["City"].unique()

cities_sql_df = pd.DataFrame({"city_name": city_unique})

cities_sql_df

,city_name
0,Berlin
1,Hamburg
2,Munich
3,Cologne
4,Düsseldorf
5,Münster
6,Hörstel


# Sending the information from notebook to SQL

In [6]:
from rupa_passwords import my_sql

In [7]:
#MySQL Connection Configuration
''' the configuration of a MySQL database connection using Python with the pymysql library.
 The provided code snippet sets up a connection string for connecting to a MySQL server with specific parameters.'''
# Define MySQL database connection parameters

# Database schema name
schema = "sql_challenge1"

# MySQL server host address
host = "127.0.0.1"

# Username for MySQL authentication
user = "root"

# Password for MySQL authentication
# Note: Replace 'my_sql' with the actual password
password = my_sql

# Port number for MySQL server
port = 3306

# Construct connection string using f-string
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'


In [13]:
cities_sql_df.to_sql('cities',
                       if_exists= 'append',
                       con= connection_string,
                       index= False)

7

**Retrieve information from SQL to this Notebook**

In [9]:
cities_from_sql = pd.read_sql("cities", con=connection_string)
cities_from_sql

,city_id,city_name
0,1,Berlin
1,2,Hamburg
2,3,Munich
3,4,Cologne
4,5,Düsseldorf
5,6,Münster
6,7,Hörstel


**Creating Population Table**

In [10]:
population= city_df[['Population', 'Year Data Retrieved']]

population['year_data'] = population['Year Data Retrieved']

population = population.drop("Year Data Retrieved", axis='columns')
population

/var/folders/yn/jzzqv3s17t5_glr9g2gqxmyh0000gn/T/ipykernel_76373/3042722708.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  population['year_data'] = population['Year Data Retrieved']


,Population,year_data
0,3576873,2024
1,1945532,2024
2,1512491,2024
3,1073096,2024
4,619477,2024
5,317713,2024
6,20506,2024


In [14]:

#sending population table to sql
population.to_sql('populations',
                  if_exists='append',
                  con=connection_string,
                  index=False)

7

In [12]:
#reading populations table from sql
pd.read_sql('populations', con=connection_string)

,city_id,population,year_data
0,1,3576873,2024
1,2,1945532,2024
2,3,1512491,2024
3,4,1073096,2024
4,5,619477,2024
5,6,317713,2024
6,7,20506,2024
